# MIT 14.75 Problem Set 5 Question 1

## Setup

In [1]:
library(stargazer)  # For tex tables.
library(lfe)  # For IV regression.
library(RCurl)  # For loading clustered SE function.


Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 

Warning message:
“package ‘lfe’ was built under R version 3.6.2”Loading required package: Matrix
Loading required package: bitops


In [2]:
load("mss_repdata.RData")

Function for clustered standard errors in `summary()`. See [this post](https://economictheoryblog.com/2018/05/18/cluster-robust-standard-errors-in-stargazer/)

In [3]:
url_robust <- "https://raw.githubusercontent.com/IsidoreBeautrelet/economictheoryblog/master/robust_summary.R"
eval(parse(text = getURL(url_robust, ssl.verifypeer = FALSE)),
     envir=.GlobalEnv)

## (a)

* Column 1: Economic growth rate ~ (growth in rainfall, t) + (growth in rainfall, t-1), clustered by country.
* Column 3: Column 1 + country fixed effects + country-specific time trends
* Column 5: Column 3 + (growth in terms of trade, t)

In [4]:
cclust <- function(m) {
    return(coef(summary(m, cluster = c("country_name")))[, 2])
}

rmse <- function(m, digits=2) {
    return(round(sqrt(mean(m$residuals^2)), digits))
}

In [5]:
fs1 = lm(gdp_g ~ GPCP_g + GPCP_g_l, data=mss_repdata)

In [6]:
fs3 = lm(gdp_g ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [7]:
fs5 = lm(gdp_g ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year + tot_100_g, data=mss_repdata)

In [8]:
stargazer(fs1, fs3, fs5,
          se=list(cclust(fs1), cclust(fs3), cclust(fs5)),
          keep=c("GPCP_g", "GPCP_g_l", "tot_100_g"),
          covariate.labels=c("Growth in rainfall, $t$",
                             "Growth in rainfall, $t-1$",
                             "Growth in terms of trade, $t+1$"),
          add.lines=list(c("Country fixed effects",
                           "no", "yes", "yes"),
                         c("Country-specific time trends",
                           "no", "yes", "yes"),
                         c("Root mean square error",
                           rmse(fs1), rmse(fs3), rmse(fs5))),
          keep.stat=c("rsq", "n"),
          dep.var.caption="Dependent Variable: Economic Growth Rate, $t$",
          dep.var.labels="Ordinary Least Squares",
          title="Rainfall and Economic Growth (First-Stage)",
          notes="Standard errors are clustered at the country level.",
          out="table2.tex",
          header=FALSE)


\begin{table}[!htbp] \centering 
  \caption{Rainfall and Economic Growth (First-Stage)} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{3}{c}{Dependent Variable: Economic Growth Rate, $t$} \\ 
\cline{2-4} 
\\[-1.8ex] & \multicolumn{3}{c}{Ordinary Least Squares} \\ 
\\[-1.8ex] & (1) & (2) & (3)\\ 
\hline \\[-1.8ex] 
 Growth in rainfall, $t$ & 0.055$^{***}$ & 0.049$^{***}$ & 0.053$^{***}$ \\ 
  & (0.016) & (0.017) & (0.018) \\ 
  & & & \\ 
 Growth in rainfall, $t-1$ & 0.034$^{***}$ & 0.028$^{**}$ & 0.037$^{**}$ \\ 
  & (0.013) & (0.014) & (0.015) \\ 
  & & & \\ 
 Growth in terms of trade, $t+1$ &  &  & $-$0.002 \\ 
  &  &  & (0.023) \\ 
  & & & \\ 
\hline \\[-1.8ex] 
Country fixed effects & no & yes & yes \\ 
Country-specific time trends & no & yes & yes \\ 
Root mean square error & 0.07 & 0.07 & 0.06 \\ 
Observations & 743 & 743 & 661 \\ 
R$^{2}$ & 0.024 & 0.133 & 0.162 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multic

Table 3

In [9]:
rf1 = lm(any_prio ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [10]:
rf2 = lm(war_prio ~ GPCP_g + GPCP_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [11]:
stargazer(rf1, rf2,
          se=list(cclust(rf1), cclust(rf2)),
          keep=c("GPCP_g", "GPCP_g_l"),
          covariate.labels=c("Growth in rainfall, $t$",
                             "Growth in rainfall, $t-1$"),
          omit=c("^(?!.*year).*factor.*$", "year"),
          perl=TRUE,
          omit.labels=c("Country fixed effects",
                        "Country-specific time trends"),
          add.lines=list(c("Root mean square error",
                           rmse(rf1), rmse(rf2))),
          keep.stat=c("rsq", "n"),
          dep.var.caption="Dependent Variable: Economic Growth Rate, $t$",
          dep.var.labels=c("Civil Conflict >=25 Deaths (OLS)",
                           "Civil Conflict >=1,000 Deaths (OLS)"),
          title="Rainfall and Civil Conflict (Reduced-Form)",
          notes="Standard errors are clustered at the country level.",
          out="table3.tex",
          header=FALSE)


\begin{table}[!htbp] \centering 
  \caption{Rainfall and Civil Conflict (Reduced-Form)} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{Dependent Variable: Economic Growth Rate, $t$} \\ 
\cline{2-3} 
\\[-1.8ex] & Civil Conflict >=25 Deaths (OLS) & Civil Conflict >=1,000 Deaths (OLS) \\ 
\\[-1.8ex] & (1) & (2)\\ 
\hline \\[-1.8ex] 
 Growth in rainfall, $t$ & $-$0.024 & $-$0.062$^{**}$ \\ 
  & (0.043) & (0.030) \\ 
  & & \\ 
 Growth in rainfall, $t-1$ & $-$0.122$^{**}$ & $-$0.069$^{**}$ \\ 
  & (0.052) & (0.032) \\ 
  & & \\ 
\hline \\[-1.8ex] 
Country fixed effects & Yes & Yes \\ 
Country-specific time trends & Yes & Yes \\ 
\hline \\[-1.8ex] 
Root mean square error & 0.24 & 0.2 \\ 
Observations & 743 & 743 \\ 
R$^{2}$ & 0.708 & 0.699 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{2}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\ 
 & \multicolumn{2}{r}{Standard errors are clustered at the country 

Column 4 and 6 of Table 4

In [12]:
ols = lm(any_prio ~ gdp_g + gdp_g_l + factor(country_name) +
         factor(country_name) * year, data=mss_repdata)

In [13]:
iv = felm(any_prio ~ country_name + country_name * year |  # Ordinary covariates
          0 |  # Factors (worked better in first term though)
          (gdp_g|gdp_g_l ~ GPCP_g + GPCP_g_l) |  # IV specification
          country_name,  # Cluster
          data=mss_repdata)

Note: The following didn't work in `stargazer` itself, so had to modify the raw TeX:
* Covariates not on the same rows between OLS and IV
* Didn't label columns correctly
* $\ge 25$ wasn't recognized

In [14]:
stargazer(ols, iv,
          se=list(cclust(ols), cclust(iv)),
          keep=c("gdp_g", "gdp_g_l"),
          covariate.labels=c("Economic growth rate, $t$",
                             "Economic growth rate, $t-1$"),
          omit=c("^(?!.*year).*factor.*$", "year"),
          perl=TRUE,
          omit.labels=c("Country fixed effects",
                        "Country-specific time trends"),
          add.lines=list(c("Root mean square error",
                           rmse(rf1), rmse(rf2))),
          keep.stat=c("rsq", "n"),
          dep.var.caption="Dependent Variable: Civil Conflict >=25 Deaths",
          dep.var.labels=c("OLS", "IV-2SLS"),  # Not working?
          title="Economic Growth and Civil Conflict",
          notes="Standard errors are clustered at the country level.",
          out="table4.tex",
          header=FALSE)


\begin{table}[!htbp] \centering 
  \caption{Economic Growth and Civil Conflict} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{Dependent Variable: Civil Conflict >=25 Deaths} \\ 
\cline{2-3} 
\\[-1.8ex] & \multicolumn{2}{c}{OLS} \\ 
\\[-1.8ex] & \textit{OLS} & \textit{felm} \\ 
\\[-1.8ex] & (1) & (2)\\ 
\hline \\[-1.8ex] 
 Economic growth rate, $t$ & $-$0.211 &  \\ 
  & (0.161) &  \\ 
  & & \\ 
 Economic growth rate, $t-1$ & 0.067 &  \\ 
  & (0.164) &  \\ 
  & & \\ 
 `gdp\_g(fit)` &  & $-$1.132 \\ 
  &  & (1.403) \\ 
  & & \\ 
 `gdp\_g\_l(fit)` &  & $-$2.546$^{**}$ \\ 
  &  & (1.102) \\ 
  & & \\ 
\hline \\[-1.8ex] 
Country fixed effects & Yes & No \\ 
Country-specific time trends & Yes & Yes \\ 
\hline \\[-1.8ex] 
Root mean square error & 0.24 & 0.2 \\ 
Observations & 743 & 743 \\ 
R$^{2}$ & 0.707 & 0.535 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:}  & \multicolumn{2}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.